<a href="https://colab.research.google.com/github/sarang-manohar/hogwarts/blob/main/academia/dataMining/code/rf_hr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone 'https://github.com/sarang-manohar/hogwarts.git'

Cloning into 'hogwarts'...
remote: Enumerating objects: 207, done.
remote: Counting objects: 100% (207/207), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 207 (delta 110), reused 81 (delta 19), pack-reused 0
Receiving objects: 100% (207/207), 23.67 MiB | 20.21 MiB/s, done.
Resolving deltas: 100% (110/110), done.


Path on git

academia/dataMining/code/rf_hr.ipynb

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, validation_curve

import warnings
warnings.filterwarnings('ignore')

In [ ]:
rd=pd.read_csv(r'/content/hogwarts/academia/dataMining/data/hrdata.csv')

display(rd)

,Unnamed: 0,employee_id,department,education,gender,no_of_trainings,age,last_promotion_rating,employee_since_years,all_tasks_completed,training_marks,is_promoted
0,0,2001001,Sales & Marketing,Master's & above,f,1,35,5,8,1,49,0
1,1,2001002,Operations,Bachelor's,m,1,30,5,4,0,60,0
2,2,2001003,Sales & Marketing,Bachelor's,m,1,34,3,7,0,50,0
3,3,2001004,Sales & Marketing,Bachelor's,m,2,39,1,10,0,50,0
4,4,2001005,Technology,Bachelor's,m,1,45,3,2,0,73,0
...,...,...,...,...,...,...,...,...,...,...,...,...
48655,54802,2049656,Sales & Marketing,Bachelor's,m,2,31,1,2,0,49,0
48656,54803,2049657,Technology,Bachelor's,m,1,48,3,17,0,78,0
48657,54804,2049658,Operations,Master's & above,f,1,37,2,6,0,56,0
48658,54805,2049659,Analytics,Bachelor's,m,1,27,5,3,1,79,0
